In [2]:
# load data
train_data = read.csv("data/c0/class_train.csv")[,-1]
train_labels = read.csv("data/c0/class_train_labels.csv")[,-1]
test_data = read.csv("data/c0/class_test.csv")[,-1]
test_labels = read.csv("data/c0/class_test_labels.csv")[,-1]

In [4]:
source("R_python_script.R")

In [6]:
fs = 12 # number of features selected with RENT

train_data_sc = apply(train_data, 2, function(x){if(length(unique(x))>1){return(scale(x))} else{return(x)}})
#train_data_sc = scale(train_data)

################ mRMR ################
library(mRMRe)
dataset = cbind("class"=train_labels, data.frame(apply(train_data_sc,2, as.numeric)))
colnames(dataset)[1] = "class"
dataset$class = factor(dataset$class, ordered=TRUE)
dataset = mRMR.data(dataset)
result = mRMR.classic(data=dataset, target_indices = c(1), feature_count = fs)

selected_features = result@filters$`1`
selected_features = colnames(dataset@data[,selected_features])
print("Performance mRMR")
print(classification_scores(train_data[,selected_features], train_labels, test_data[,selected_features], test_labels))

############### Laplace Score #######################
library(Rdimtools)
lapl = do.lscore(as.matrix(train_data_sc), t=10, ndim = fs) # train_data must be matrix
selected_features = lapl$featidx
print("Performance L-score")
print(classification_scores(train_data[,selected_features], train_labels, test_data[,selected_features], test_labels))

############### Fisher Score ########################
fscore = do.fscore(as.matrix(train_data_sc), ndim = fs,label = train_labels)
selected_features = fscore$featidx
print("Performance F-score")
print(classification_scores(train_data[,selected_features], train_labels, test_data[,selected_features], test_labels))


[1] "Performance mRMR"
$f1_1
[1] 0.7532468

$f1_0
[1] 0.739726

$acc
[1] 0.7466667

$mcc
[1] 0.4953682

[1] "Performance L-score"
$f1_1
[1] 0.7160494

$f1_0
[1] 0.6666667

$acc
[1] 0.6933333

$mcc
[1] 0.3829644

[1] "Performance F-score"
$f1_1
[1] 0.7466667

$f1_0
[1] 0.7466667

$acc
[1] 0.7466667

$mcc
[1] 0.5



In [7]:
source("stabsel_gridsearch.R")

Loading required package: parallel

Loading required package: Matrix

Loaded glmnet 4.1-2

Loading required package: lattice

Loading required package: ggplot2


Attaching package: 'caret'


The following object is masked from 'package:survival':

    cluster




In [ ]:
selected_features = stabsel_gridsearch(train_data, train_labels)

In [ ]:
classification_scores(train_data[,selected_features], train_labels, test_data[,selected_features], test_labels)